In [15]:
import sys
import random

from typing import Dict, List

import matplotlib.pyplot as plt
import numpy
import torch
from torch import nn
from torch.nn import functional as F


In [16]:
import pathlib
from pathlib import Path


current_dir: Path = pathlib.Path().cwd().resolve()
project_root: Path = current_dir.parent
data_dir: Path = project_root / "data"

sys.path.append(str(project_root))


In [17]:
from corsmal_challenge.utils import fix_random_seeds, behave_deterministically

fix_random_seeds(0)
behave_deterministically()

In [18]:
import json


with open(str(data_dir / "train" / "ccm_train_annotation.json"), "r") as f:
    dic = json.load(f)
    annotations: Dict = dic["annotations"]

    type_label = {
        0: "no content",
        1: "pasta",
        2: "rice",
        3: "water",
    }
    level_label = {
        0: "0%",
        1: "50%",
        2: "90%",
    }

    classified_data = [[[] for j in range(3)] for i in range(4)]
    for data in annotations:
        id = data["id"]
        filling_type = data["filling type"]
        filling_level = data["filling level"]
        classified_data[filling_type][filling_level].append(id)
    # print(classified_data)

    for i, d in enumerate(classified_data):
        for j, lis in enumerate(d):
            print(type_label[i], level_label[j])
            print(lis)


no content 0%
[1, 3, 8, 12, 14, 17, 36, 41, 42, 46, 49, 54, 66, 77, 82, 89, 90, 99, 114, 115, 119, 128, 148, 150, 151, 154, 156, 161, 178, 180, 195, 196, 202, 205, 213, 217, 220, 222, 245, 254, 260, 263, 269, 285, 291, 296, 298, 312, 320, 326, 329, 349, 350, 351, 361, 363, 368, 381, 385, 388, 393, 400, 406, 410, 417, 419, 426, 428, 430, 439, 445, 448, 455, 460, 468, 473, 474, 484, 485, 499, 512, 519, 522, 525, 540, 542, 544, 548, 552, 554, 570, 588, 593, 595, 609, 614, 618, 620, 622, 634, 637, 645, 652, 654, 656, 673, 678, 683]
no content 50%
[]
no content 90%
[]
pasta 0%
[]
pasta 50%
[4, 13, 15, 24, 28, 30, 31, 33, 40, 51, 55, 59, 75, 78, 86, 88, 91, 94, 97, 106, 112, 124, 127, 132, 136, 140, 142, 155, 171, 176, 181, 182, 188, 189, 190, 194, 198, 200, 201, 224, 226, 227, 228, 229, 232, 234, 239, 257, 276, 277, 284, 288, 290, 293, 311, 321, 323, 324, 327, 330, 345, 354, 362, 367, 389, 392, 397, 411, 414, 418, 425, 432, 436, 441, 442, 443, 452, 457, 459, 461, 462, 492, 493, 496, 497, 50

In [19]:
# divide (train : val == 80 : 20)
val_ids = []
train_ids = []
for i, d in enumerate(classified_data):
    for j, lis in enumerate(d):
        val_list = set(random.sample(lis, len(lis) // 10 * 2))
        for id in lis:
            if id in val_list:
                val_ids.append(id)
            else:
                train_ids.append(id)

print(val_ids)
print(train_ids)


[17, 66, 99, 161, 205, 220, 245, 296, 326, 349, 351, 400, 406, 417, 419, 468, 618, 620, 637, 678, 51, 75, 97, 188, 224, 226, 228, 234, 321, 345, 414, 432, 457, 459, 461, 503, 547, 566, 631, 636, 19, 22, 60, 85, 167, 225, 366, 396, 435, 458, 489, 535, 539, 573, 584, 598, 643, 658, 664, 670, 35, 45, 57, 111, 175, 212, 231, 235, 282, 305, 307, 370, 404, 434, 449, 486, 541, 585, 633, 657, 76, 87, 100, 165, 209, 214, 215, 246, 306, 359, 387, 399, 456, 465, 511, 520, 524, 589, 667, 672, 53, 70, 110, 219, 248, 287, 294, 338, 356, 373, 429, 528, 594, 655, 50, 120, 162, 191, 262, 264, 267, 274, 303, 331, 384, 472, 543, 650]
[1, 3, 8, 12, 14, 36, 41, 42, 46, 49, 54, 77, 82, 89, 90, 114, 115, 119, 128, 148, 150, 151, 154, 156, 178, 180, 195, 196, 202, 213, 217, 222, 254, 260, 263, 269, 285, 291, 298, 312, 320, 329, 350, 361, 363, 368, 381, 385, 388, 393, 410, 426, 428, 430, 439, 445, 448, 455, 460, 473, 474, 484, 485, 499, 512, 519, 522, 525, 540, 542, 544, 548, 552, 554, 570, 588, 593, 595, 609,

In [20]:
from corsmal_challenge.data.dataset import AudioDataset


ds = AudioDataset(
    data_dir / "train",
    data_dir / "train" / "ccm_train_annotation.json",
    seed=0,
    train=True,
    query="type",
    random_crop=False,
)


In [21]:
from corsmal_challenge.data.data_loader import ReproducibleDataLoader as DataLoader


dl = DataLoader(ds, 0, shuffle=True)

for i, (d, l) in enumerate(dl):
    print(i, l,d.size())


0 tensor([2]) torch.Size([1, 8, 128, 123])
1 tensor([2]) torch.Size([1, 8, 128, 390])
2 tensor([3]) torch.Size([1, 8, 128, 539])
3 tensor([3]) torch.Size([1, 8, 128, 496])
4 tensor([2]) torch.Size([1, 8, 128, 109])
5 tensor([1]) torch.Size([1, 8, 128, 554])
6 tensor([2]) torch.Size([1, 8, 128, 372])
7 tensor([1]) torch.Size([1, 8, 128, 179])
8 tensor([0]) torch.Size([1, 8, 128, 120])
9 tensor([3]) torch.Size([1, 8, 128, 676])
10 tensor([0]) torch.Size([1, 8, 128, 70])
11 tensor([0]) torch.Size([1, 8, 128, 114])
12 tensor([1]) torch.Size([1, 8, 128, 278])
13 tensor([1]) torch.Size([1, 8, 128, 331])
14 tensor([1]) torch.Size([1, 8, 128, 103])
15 tensor([3]) torch.Size([1, 8, 128, 430])
16 tensor([1]) torch.Size([1, 8, 128, 130])
17 tensor([1]) torch.Size([1, 8, 128, 177])
18 tensor([2]) torch.Size([1, 8, 128, 969])
19 tensor([3]) torch.Size([1, 8, 128, 560])
20 tensor([3]) torch.Size([1, 8, 128, 317])
21 tensor([1]) torch.Size([1, 8, 128, 143])
22 tensor([3]) torch.Size([1, 8, 128, 555])